#### _analysis for  the TrackSelectorByRegion_

### Read the HLT-Report 
*HLTReport*
  * path name
    * WasRun
    * L1S
    * Pre
    * HLT
    * %L1sPre
    * Rate
    * RateHi
    * Errors

In [ ]:
def getHLTReport(fname):
    f=open('logNoCust','r')
    l=f.readline()
    hltReportDict={'WasRun':0.0,'L1S':0.0,'Pre':0.0,'HLT':0.0,'%L1sPre':0,'Rate':0.0,'RateHi':0.0, 'Errors':0}
    hltReportKeys=list(hltReportDict.keys())
    HLTReport={}
    SummaryFlag=False
    while l:
        if 'HLT-Report' not in l:
            l=f.readline()
            continue
        if '---------- HLTrig Summary ------------' in l:
            SummaryFlag=True
            l=f.readline()
            l=f.readline()
            continue
        if 'end!' in l:
            SummaryFlag=False
            break
        if SummaryFlag:
            currdict=hltReportDict.copy()
            items=l[:-1].split(' ')
            while '' in items:
                items.remove('')
            items.pop(0)
            items.pop(0)
            for i in range(len(hltReportKeys)):
                currdict[hltReportKeys[i]]=float(items[i])
            HLTReport.update({items[-1]:currdict})
        l=f.readline()
    f.close()
    return HLTReport

### Read the Time Report
 
*TimeReport*
  * pathSummary
    * perEvent
    * perExec
    
  * pathDetails 
    * path_name
      * perEvent 
      * perVisit
      * Name
      
  * moduleSummary 
    * module_name
      * perEvent
      * perExec
      * perVisit  
    

In [ ]:
def getTimeReport(fname):
    f=open(fname,'r')
    l=f.readline()

    timeReportDict={'perEvent':0.0,'perExec':0.0}
    timeReportKeys=list(timeReportDict.keys())

    timeReportDetailsDict={'perEvent':[],'perVisit':[],'Name':[]}
    timeReportDetailsDictKeys=list(timeReportDetailsDict.keys())

    timeReportModulesDict={'perEvent':[],'perExec':[],'perVisit':[]}
    timeReportModulesDictKeys=list(timeReportModulesDict.keys())

    TimeReport={'pathSummary':{},'pathDetails':{},'moduleSummary':{}}
    SummaryFlag=False
    DetailsFlag=False
    ModulesFlag=False
    while l:
        if 'TimeReport' not in l:
            SummaryFlag=False
            DetailsFlag=False
            ModulesFlag=False
            l=f.readline()
            continue
        if 'per event    per visit  Name' in l :
            l=f.readline()
            continue
    #     print(l[:-1])
        if '---------- Path   Summary ---' in l:
            SummaryFlag=True
            l=f.readline()
            l=f.readline()
            continue
        if '---------- Module Summary --' in l:
            ModulesFlag=True
            l=f.readline()
            l=f.readline()
            continue

        if '---------- Modules in Path:' in l:
            DetailsFlag=True
            l=l[:-1].replace("---------- Modules in Path: ","")
            l=l.replace(" ---[Real sec]----","")
            currPath=l.replace("TimeReport ","")
            TimeReport['pathDetails'].update({currPath:timeReportDetailsDict.copy()})
    #         print(l)
            l=f.readline()
            l=f.readline()
            continue

        ## bug fix
        if 'per event ' in l:
            SummaryFlag=False
            ModulesFlag=False

        if SummaryFlag:
            currdict=timeReportDict.copy()
            items=l[:-1].split(' ')
            while '' in items:
                items.remove('')
            items.pop(0)
    #         print(items)
            for i in range(len(timeReportKeys)):
                currdict[timeReportKeys[i]]=float(items[i])
            TimeReport['pathSummary'].update({items[-1]:currdict})

        if DetailsFlag:
    #         print(l[:-1])
            items=l[:-1].split(' ')
            while '' in items:
                items.remove('')
            items.pop(0)
            for i in range(len(timeReportDetailsDictKeys)-1):
                TimeReport['pathDetails'][currPath][timeReportDetailsDictKeys[i]].append(float(items[i]))
            TimeReport['pathDetails'][currPath][timeReportDetailsDictKeys[-1]].append(items[i])

        if ModulesFlag:
            currdict=timeReportModulesDict.copy()
            items=l[:-1].split(' ')
            while '' in items:
                items.remove('')
            items.pop(0)
    #         print(l[:-1])
    #         print(items)
            for i in range(len(timeReportModulesDictKeys)):
                currdict[timeReportModulesDictKeys[i]]=float(items[i])
            TimeReport['moduleSummary'].update({items[-1]:currdict})

        l=f.readline()
    f.close()
    return TimeReport

#### Analysis

In [ ]:
origHLTReport=getHLTReport('logNoCust')
origTimeReport=getTimeReport('logNoCust')
print(" total number of  paths  = ",len(HLTReport))
print(" total number of  paths in pathSummary  = ",len(origTimeReport['pathSummary']))
print(" total number of  paths in pathDetails  = ",len(origTimeReport['pathDetails']))
print(" total number of  modules in moduleSummary  = ",len(origTimeReport['moduleSummary']))

In [ ]:
custHLTReport=getHLTReport('logWithMuonCust')
custTimeReport=getTimeReport('logWithMuonCust')
print(" total number of  paths  = ",len(HLTReport))
print(" total number of  paths in pathSummary  = ",len(custTimeReport['pathSummary']))
print(" total number of  paths in pathDetails  = ",len(custTimeReport['pathDetails']))
print(" total number of  modules in moduleSummary  = ",len(custTimeReport['moduleSummary']))

In [ ]:
origModuleList=list(origTimeReport['moduleSummary'].keys())
custModuleList=list(custTimeReport['moduleSummary'].keys())
deletedModules=[]
swappedModules=[]

i=1
print("deleted modules ")
for om in origModuleList:
    if om not in custModuleList:
        print(i,"\t",om)
        deletedModules.append(om)
        i+=1
f=open('swappedModules.txt','r')
l=f.readline()
print("\nswapped modules ")
i=1
while l:
    swappedModules.append(l[:-1])
    print(i,"\t",l[:-1])
    i+=1
    l=f.readline()
f.close()


In [ ]:
perEvt={'orig':[],'cust':[]}
perVisit={'orig':[],'cust':[]}
perExec={'orig':[],'cust':[]}

for modu in deletedModules:
    perEvt['orig'].append(origTimeReport['moduleSummary'][modu]['perEvent'])
    perVisit['orig'].append(origTimeReport['moduleSummary'][modu]['perVisit'])
    perExec['orig'].append(origTimeReport['moduleSummary'][modu]['perExec'])

    perEvt['cust'].append(0.0)
    perVisit['cust'].append(0.0)
    perExec['cust'].append(0.0)

for modu in swappedModules:
    if modu=='hltIter0HighPtTkMuPixelTracks':
        print("got it")
        continue
    perEvt['orig'].append(origTimeReport['moduleSummary'][modu]['perEvent'])
    perVisit['orig'].append(origTimeReport['moduleSummary'][modu]['perVisit'])
    perExec['orig'].append(origTimeReport['moduleSummary'][modu]['perExec'])

    perEvt['cust'].append(custTimeReport['moduleSummary'][modu]['perEvent'])
    perVisit['cust'].append(custTimeReport['moduleSummary'][modu]['perVisit'])
    perExec['cust'].append(custTimeReport['moduleSummary'][modu]['perExec'])

In [ ]:
import numpy as np

In [ ]:
print(np.round(sum(perEvt['orig'])*1000,3),np.round(sum(perEvt['cust'])*1000,3))
print(np.round(sum(perVisit['orig'])*1000,3),np.round(sum(perVisit['cust'])*1000,3))
print(np.round(sum(perExec['orig'])*1000,3),np.round(sum(perExec['cust'])*1000,3))

#### Results for the Muon Reco using TrackSelectorByRegion
* PerEvent 0.667  -> 0.023
* PerVisit 0.441  -> 0.018
* PerExec  1.058  -> 0.039

### Read Json from FastTimeService

In [ ]:
import json
import numpy as np

In [ ]:
with open('TTBarNoCustResourcesSat22Aug.json') as f:
    origFTS = json.load(f)
with open('TTBarAllButRegClusSat22Aug.json') as f:
    custFTS = json.load(f)
origModuleList=[ modu['label'] for modu in origFTS['modules'] ]
custModuleList=[ modu['label'] for modu in custFTS['modules'] ]

In [ ]:
f=open('pixelTracks.txt','w')
for modu in origFTS['modules']:
    if 'PixelTrackProducer' in modu['type']:
#         print (modu['label'], "\t",modu['type'])
        f.write(modu['label']+"\t"+modu['type']+'\n')
f.close()

f=open('trackingRegions.txt','w')
for modu in origFTS['modules']:
    if 'TrackingRegion' in modu['label']:
#         print (modu['label'], "\t",modu['type'])
        f.write(modu['label']+"\t"+modu['type']+'\n')
f.close()

In [ ]:
swappedModules=[]
f=open('swappedModules.txt','r')
l=f.readline()
print("\nswapped modules ")
i=1
while l:
    swappedModules.append(l[:-1])
    print(i,"\t",l[:-1])
    i+=1
    l=f.readline()
f.close()

i=1
deletedModules=[]
print("\nDeleted modules")
for module in origModuleList:
    if module not in custModuleList:
        deletedModules.append(module)
        print(i,"\t",module)
        i+=1
    

In [ ]:
# with open('ZMuMuNoCustResources.json') as f:
#     origFTS = json.load(f)
# with open('AllButRegClustersResources.json') as f:
#     custFTS = json.load(f)
# origModuleList=[ modu['label'] for modu in origFTS['modules'] ]
# custModuleList=[ modu['label'] for modu in custFTS['modules'] ]
# swappedModules=[]
# f=open('swappedModules.txt','r')
# l=f.readline()
# print("\nswapped modules ")
# i=1
# while l:
#     swappedModules.append(l[:-1])
#     print(i,"\t",l[:-1])
#     i+=1
#     l=f.readline()
# f.close()

# i=1
# deletedModules=[]
# print("\nDeleted modules")
# for module in origModuleList:
#     if module not in custModuleList:
#         deletedModules.append(module)
#         print(i,"\t",module)
#         i+=1
    

In [ ]:
origTime={}
custTime={}
origVal=0.0
custVal=0.0
idx='time_thread'
for modu in origFTS['modules']:
    if modu['label'] in swappedModules:
        origTime.update({modu['label']:modu[idx]})
        origVal+=modu[idx]
        
for modu in custFTS['modules']:
    if modu['label'] in swappedModules:
        custTime.update({modu['label']:modu[idx]})        
        custVal+=modu[idx]
    
for key in origTime.keys():        
    print(key,"\t",np.round(origTime[key],3),"\t",np.round(custTime[key],3))

for modu in origFTS['modules']:    
    if modu['label'] in deletedModules:
        print(modu['label'],"\t",np.round(modu[idx],3),"\t0.0 ")
        origVal+=modu[idx]
print("Total Original value   : ",origVal)
print("Total Customized value : ",custVal)

In [ ]:
def printDetails(jdata,mlabel):
    for modu in jdata['modules']:
        if modu['label'] ==mlabel:
            for k in modu:
                print(k," : ",modu[k])
            break
    

In [ ]:
printDetails(origFTS,'hltIter1ClustersRefRemoval')